In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, avg, dayofweek, expr, hour, month, to_date, count

Create Spark Session

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") 
    .config('spark.executor.memoryOverhead', '3000')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 17:28:09 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/19 17:28:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 17:28:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Load Preprocessed Taxi Data - 3 months segments

In [3]:
# Read all Parquet files in the specified folder
sdf_yellow_2016_01_to_03 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2016_01_to_03.parquet/*')
sdf_yellow_2016_04_to_06 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2016_04_to_06.parquet/*')
sdf_yellow_2016_07_to_09 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2016_07_to_09.parquet/*')
sdf_yellow_2016_10_to_12 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2016_10_to_12.parquet/*')

sdf_yellow_2017_01_to_03 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2017_01_to_03.parquet/*')
sdf_yellow_2017_04_to_06 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2017_04_to_06.parquet/*')
sdf_yellow_2017_07_to_09 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2017_07_to_09.parquet/*')
sdf_yellow_2017_10_to_12 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2017_10_to_12.parquet/*')

sdf_yellow_2018_01_to_03 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2018_01_to_03.parquet/*')
sdf_yellow_2018_04_to_06 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2018_04_to_06.parquet/*')
sdf_yellow_2018_07_to_09 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2018_07_to_09.parquet/*')
sdf_yellow_2018_10_to_12 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2018_10_to_12.parquet/*')

sdf_yellow_2019_01_to_03 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2019_01_to_03.parquet/*')
sdf_yellow_2019_04_to_06 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2019_04_to_06.parquet/*')
sdf_yellow_2019_07_to_09 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2019_07_to_09.parquet/*')
sdf_yellow_2019_10_to_12 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2019_10_to_12.parquet/*')

# List creation of all taxi data
taxi_periods = [
    sdf_yellow_2016_01_to_03,
    sdf_yellow_2016_04_to_06,
    sdf_yellow_2016_07_to_09,
    sdf_yellow_2016_10_to_12,
    sdf_yellow_2017_01_to_03,
    sdf_yellow_2017_04_to_06,
    sdf_yellow_2017_07_to_09,
    sdf_yellow_2017_10_to_12,
    sdf_yellow_2018_01_to_03,
    sdf_yellow_2018_04_to_06,
    sdf_yellow_2018_07_to_09,
    sdf_yellow_2018_10_to_12,
    sdf_yellow_2019_01_to_03,
    sdf_yellow_2019_04_to_06,
    sdf_yellow_2019_07_to_09,
    sdf_yellow_2019_10_to_12,
]

Create weekend column

In [4]:
# List to store modified DataFrames with the "weekend" column
modified_taxi_periods = []

# Loop through the list of DataFrames and add the "weekend" column
for taxi_df in taxi_periods:
    modified_taxi_df = taxi_df.withColumn("Weekend", expr("(dayofweek(tpep_pickup_datetime) = 1) OR (dayofweek(tpep_pickup_datetime) = 7)").cast("int"))
    modified_taxi_periods.append(modified_taxi_df)

Create pick up hour and month column

In [5]:
# List to store modified DataFrames with the "month" and "hour" columns
timed_taxi_periods = []

# Loop through the list of DataFrames and add the "month" and "hour" columns
for taxi_df in modified_taxi_periods:
    full_modified_taxi_df = taxi_df.withColumn("Month", month(col("tpep_pickup_datetime")))
    full_modified_taxi_df = full_modified_taxi_df.withColumn("Hour", hour(col("tpep_pickup_datetime")))
    timed_taxi_periods.append(full_modified_taxi_df)

Create date column

In [6]:
# Assuming timed_taxi_periods is your DataFrame list
dated_taxi_periods = []

for df in timed_taxi_periods:
    # Add a new column "Date" based on "tpep_pickup_datetime"
    df_with_date = df.withColumn("Date", to_date(col("tpep_pickup_datetime")))
    dated_taxi_periods.append(df_with_date)

Aggregate taxi data frame

In [7]:
# Assuming `selected_dataframes` is your list of DataFrames
# You can loop through each DataFrame in the list and perform aggregations
taxi_hourly_dataframes = []
for dataframe in dated_taxi_periods:
    # Select the specified columns to keep
    selected_columns = [
        "Hour", "Date", "PULocationID", "Weekend", "Month"
    ]
    
    # Group by pickup_hour, LocationID
    grouped_data = dataframe.groupBy(*selected_columns).agg(
        count("*").alias("number_of_trips"),
    )
    
    # Append to aggregated_dataframes list
    taxi_hourly_dataframes.append(grouped_data)

Trying to save data

In [9]:
# Define a list of output paths for each 3-month block
three_month_blocks = [
    "2016_01_to_03", "2016_04_to_06", "2016_07_to_09", "2016_10_to_12",
    "2017_01_to_03", "2017_04_to_06", "2017_07_to_09", "2017_10_to_12",
    "2018_01_to_03", "2018_04_to_06", "2018_07_to_09", "2018_10_to_12",
    "2019_01_to_03", "2019_04_to_06", "2019_07_to_09", "2019_10_to_12"
]

# Iterate through each DataFrame and its corresponding 3-month block
for i, dataframe in enumerate(taxi_hourly_dataframes):
    block_name = three_month_blocks[i]
    output_path = f"../data/raw/yellow_taxi_hourly/{block_name}.parquet"
    dataframe.write.parquet(output_path)


Size of Taxi Dataframe

In [28]:
# Function to count number of rows in data
def calculate_total_rows(dataframes):
    total_rows = 0
    for df in dataframes:
        total_rows += df.count()
    return total_rows

size_of_data = calculate_total_rows(taxi_hourly_dataframes)
print("Size of aggregated hourly dataframes:", size_of_data)


# Stop the SparkSession
spark.stop()

Size of initial import: 4344263
